# Model calculation

This notebook obtains an aggregate model and a best individual classification for each model in galaxy builder's original 296 subjects.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import numpy as np
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
from tqdm import tqdm
import lib.galaxy_utilities as gu
import gzbuilder_analysis.parsing as parsing
import gzbuilder_analysis.rendering as rendering
import gzbuilder_analysis.aggregation as aggregation
import gzbuilder_analysis.fitting as fitting
from gzbuilder_analysis.fitting.jupyter import UpdatableDisplay
from IPython.display import HTML

Parameters of input (or output) file locations, and some opertational flags

In [3]:
AGGREGATE_LOCATION = 'lib/aggregation_results.pickle'
BEST_INDIVIDUAL_LOCATION = 'lib/best_individual.pickle'

Load in the list of subject ids to use

In [4]:
sid_list = np.loadtxt('lib/subject-id-list.csv', dtype='u8')
gal_angle_df = pd.read_csv('lib/gal-metadata.csv', index_col=0)
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

## Preparation

Define functions to obtain the best individual classification for a galaxy, and its aggregate model

In [5]:
def get_agg_model(subject_id):
  gal = gal_angle_df.loc[subject_id]
  angle = gal['angle']
  diff_data = diff_data_df.loc[subject_id]
  size_diff = diff_data['size_diff']
  cls = gu.classifications.query(
    'subject_ids == {}'.format(subject_id)
  )
  model, error, masks, arms = aggregation.make_model(
    cls, gal, angle
  )
  scaled_model = parsing.scale_aggregate_model(model, size_diff=size_diff)
  scaled_errors = parsing.scale_model_errors(error, size_diff=size_diff)
  return scaled_model, scaled_errors, masks, arms
  
def get_best_cls(subject_id):
  gal = gal_angle_df.loc[subject_id]
  angle = gal['angle']
  diff_data = diff_data_df.loc[subject_id]

  ba = gal['PETRO_BA90']
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  size_diff = diff_data['size_diff']


  cls = gu.classifications.query(
    'subject_ids == {}'.format(subject_id)
  )
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(parsing.parse_annotation, size_diff=size_diff)
  rendered = models.apply(
    rendering.calculate_model,
    image_size=galaxy_data.shape[0],
    psf=psf
  )
  
  losses = rendered.apply(fitting.loss, args=(galaxy_data,), pixel_mask=pixel_mask)
  return losses.idxmin(), models.loc[losses.idxmin()]

Calculate the list of best individual models

In [6]:
best_indiv = pd.Series([])
with tqdm(sid_list, desc='Finding best individual', leave=True) as bar:
  for subject_id in bar:
    best_indiv.loc[subject_id] = get_best_cls(subject_id)
best_indiv = best_indiv.apply(pd.Series)
best_indiv.columns = ('Index', 'Model')
best_indiv.to_pickle(BEST_INDIVIDUAL_LOCATION)

Finding best individual:   0%|          | 0/296 [00:00<?, ?it/s]

Finding best individual:   0%|          | 1/296 [00:08<40:44,  8.29s/it]

Finding best individual:   1%|          | 2/296 [00:19<44:13,  9.02s/it]

Finding best individual:   1%|          | 3/296 [00:31<49:27, 10.13s/it]

Finding best individual:   1%|▏         | 4/296 [00:37<42:57,  8.83s/it]

...

Finding best individual:  99%|█████████▊| 292/296 [46:31<00:36,  9.17s/it]

Finding best individual:  99%|█████████▉| 293/296 [46:36<00:24,  8.09s/it]

Finding best individual:  99%|█████████▉| 294/296 [46:38<00:12,  6.11s/it]

Finding best individual: 100%|█████████▉| 295/296 [46:39<00:04,  4.77s/it]

Finding best individual: 100%|██████████| 296/296 [46:43<00:00,  4.36s/it]

In [7]:
best_indiv

,Index,Model
20901989,7582,"{'disk': {'mu': [32.534912109375, 32.5390625],..."
20902000,8746,"{'disk': {'mu': [90.19638061523438, 90.1840209..."
20902007,5745,"{'disk': {'mu': [89.22597867250443, 88.0966779..."
20902023,9462,"{'disk': {'mu': [74.94947856664658, 76.0800154..."
20902031,2002,"{'disk': {'mu': [80.5333251953125, 78.87945556..."
20902048,1206,"{'disk': {'mu': [43.26777309179306, 44.0041029..."
20902059,9841,"{'disk': {'mu': [92.6253662109375, 93.36328125..."
20902062,2171,"{'disk': {'mu': [93.27083259820938, 92.1266250..."
20902073,7139,"{'disk': {'mu': [35.0528963804245, 39.63281702..."
20902086,4486,"{'disk': {'mu': [50.139723122119904, 51.853604..."


Calculate the list of aggregate models

In [8]:
tqdm.pandas(desc='Calculating aggregate', leave=True)
aggregation_results = pd.Series(
  sid_list, index=sid_list
).progress_apply(get_agg_model)\
  .apply(pd.Series)
aggregation_results.columns = ('Model', 'Errors', 'Masks', 'Arms')
aggregation_results.to_pickle(AGGREGATE_LOCATION)

Calculating aggregate:   0%|          | 0/296 [00:00<?, ?it/s]

Calculating aggregate:   1%|          | 2/296 [00:05<13:32,  2.76s/it]

Calculating aggregate:   1%|          | 3/296 [00:11<18:24,  3.77s/it]

Calculating aggregate:   1%|▏         | 4/296 [00:21<27:21,  5.62s/it]

Calculating aggregate:   2%|▏         | 5/296 [00:25<24:29,  5.05s/it]

...

Calculating aggregate:  99%|█████████▊| 292/296 [38:52<00:29,  7.27s/it]

Calculating aggregate:  99%|█████████▉| 293/296 [38:57<00:20,  6.70s/it]

Calculating aggregate:  99%|█████████▉| 294/296 [39:06<00:14,  7.28s/it]

Calculating aggregate: 100%|█████████▉| 295/296 [39:12<00:07,  7.17s/it]

Calculating aggregate: 100%|██████████| 296/296 [39:19<00:00,  7.05s/it]